In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.splits import create_splits_scenes
from nuscenes.can_bus.can_bus_api import NuScenesCanBus

import sys
sys.path.append('/mnt/cache/wangxiaodong/SDM/src')

In [2]:
DATAROOT = '/mnt/lustrenew/wangxiaodong/data/nuscene'

class frameDataset():
    def __init__(self, split='val'):
        self.split = split

        self.nusc = NuScenes(version='v1.0-trainval', dataroot=DATAROOT, verbose=True)

        self.splits = create_splits_scenes()

        # training samples
        self.samples = self.get_samples(split)
        print('Total samples: %d' % len(self.samples))
        self.samples_groups = self.group_sample_by_scene(split)
    def __len__(self):
        return len(self.samples)
    
    def get_samples(self, split='train'):
        scenes = self.splits[split]
        samples = [samp for samp in self.nusc.sample if
                   self.nusc.get('scene', samp['scene_token'])['name'] in scenes]
        return samples
    def group_sample_by_scene(self, split='train'):
        scenes = self.splits[split]
        samples_dict = {}
        for sce in scenes:
            samples_dict[sce] = [] # empty list
        for samp in self.nusc.sample:
            scene_token = samp['scene_token']
            scene = self.nusc.get('scene', scene_token)
            tmp_sce = scene['name']
            if tmp_sce in scenes:
                samples_dict[tmp_sce].append(samp)
        return samples_dict

In [3]:
dataset = frameDataset('val')
scenes_list = list(dataset.samples_groups.keys())
nusc_can = NuScenesCanBus(dataroot=DATAROOT)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 36.724 seconds.
Reverse indexing ...
Done reverse indexing in 9.2 seconds.
Total samples: 6019


In [4]:
def get_sample_utime(token):
    sample_path = dataset.nusc.get('sample_data', token)['filename']
    sample_utime = int(sample_path.split('__')[-1].split('.')[0])
    return sample_utime

def get_sample_filename(token):
    sample_path = dataset.nusc.get('sample_data', token)['filename']
    return sample_path

def sort_samples(unsort_samples):
    samples_with_utime = []

    for item in unsort_samples:
        token = item['data']['CAM_FRONT']
        sample_utime = get_sample_utime(token)
        samples_with_utime.append((sample_utime, item))

    samples_with_utime.sort(key=lambda x: x[0])

    sorted_samples = [item for _, item in samples_with_utime]

    return sorted_samples

In [5]:
import bisect

def interpolate_speed(veh_speed, timestamp):
    if timestamp < veh_speed[0]['utime']:
        return veh_speed[0]['vehicle_speed']
    # 找到timestamp在veh_speed中的位置
    index = bisect.bisect_left([d['utime'] for d in veh_speed], timestamp)
    
    # 如果timestamp在veh_speed的最后一个元素之后，则返回最后一个元素的速度
    if index == len(veh_speed):
        return veh_speed[-1]['vehicle_speed']
    
    # 如果timestamp正好等于某个utime，则直接返回该utime对应的速度
    if veh_speed[index]['utime'] == timestamp:
        return veh_speed[index]['vehicle_speed']
    
    # 如果timestamp在两个utime之间，则进行插值
    prev_utime = veh_speed[index - 1]['utime']
    next_utime = veh_speed[index]['utime']
    prev_speed = veh_speed[index - 1]['vehicle_speed']
    next_speed = veh_speed[index]['vehicle_speed']
    
    # 进行线性插值
    interpolated_speed = prev_speed + ((next_speed - prev_speed) / (next_utime - prev_utime)) * (timestamp - prev_utime)
    
    return interpolated_speed

def find_nearest_speed(veh_speed, sorted_samples):
    result = []
    for sample in sorted_samples:
        timestamp = sample['timestamp']
        if timestamp > veh_speed[-1]['utime']:
            continue
        nearest_speed = interpolate_speed(veh_speed, timestamp)
        result.append(nearest_speed)
    return result

In [6]:
import bisect

def interpolate_angle(veh_angles, timestamp):
    if timestamp < veh_angles[0]['utime']:
        return veh_angles[0]['value']
    # 找到timestamp在veh_speed中的位置
    index = bisect.bisect_left([d['utime'] for d in veh_angles], timestamp)
    
    # 如果timestamp在veh_speed的最后一个元素之后，则返回最后一个元素的速度
    if index == len(veh_angles):
        return veh_angles[-1]['value']
    
    # 如果timestamp正好等于某个utime，则直接返回该utime对应的速度
    if veh_angles[index]['utime'] == timestamp:
        return veh_angles[index]['value']
    
    # 如果timestamp在两个utime之间，则进行插值
    prev_utime = veh_angles[index - 1]['utime']
    next_utime = veh_angles[index]['utime']
    prev_angle = veh_angles[index - 1]['value']
    next_angle = veh_angles[index]['value']
    
    # 进行线性插值
    interpolated_angle = prev_angle + ((next_angle - prev_angle) / (next_utime - prev_utime)) * (timestamp - prev_utime)
    
    return interpolated_angle

def find_nearest_angle(veh_angles, sorted_samples):
    result = []
    for sample in sorted_samples:
        timestamp = sample['timestamp']
        if timestamp > veh_angles[-1]['utime']:
            continue
        nearest_speed = interpolate_angle(veh_angles, timestamp)
        result.append(nearest_speed)
    return result

In [7]:
# extract angle and speed for 40 frames in all scene
# val
meta_data = []
for sce in scenes_list:
    unsort_samples = dataset.samples_groups[sce]
    sorted_samples = sort_samples(unsort_samples)
    veh_angle = nusc_can.get_messages(sce, 'steeranglefeedback')
    veh_speed = nusc_can.get_messages(sce, 'vehicle_monitor')

    angles = find_nearest_angle(veh_angle, sorted_samples)
    speeds = find_nearest_speed(veh_speed, sorted_samples)
    # print(f'scene: {sce}, len of angles: {len(angles)}, len of speeds: {len(speeds)}')
    if len(angles)!= len(speeds):
        print(f'scene: {sce}, len of angles: {len(angles)}, len of speeds: {len(speeds)}')
    
    filenames = []
    for sample in sorted_samples:
        token = sample['data']['CAM_FRONT']
        sample_filename = get_sample_filename(token)
        filenames.append(sample_filename)

    meta_data.append(
        {
            'scene': sce,
            'files': filenames,
            'angles': angles,
            'speeds': speeds
        }
    )
len(meta_data)
import json
with open('/mnt/lustrenew/wangxiaodong/data/nuscene/scene_action_file_val.json', 'w') as f:
    json.dump(meta_data, f)

scene: scene-0100, len of angles: 39, len of speeds: 40
scene: scene-0273, len of angles: 38, len of speeds: 39
scene: scene-0562, len of angles: 39, len of speeds: 40


In [8]:
# extract angle and speed for 40 frames in all scene
# train
meta_data = []
for sce in scenes_list:
    unsort_samples = dataset.samples_groups[sce]
    sorted_samples = sort_samples(unsort_samples)
    try:
        veh_angle = nusc_can.get_messages(sce, 'steeranglefeedback')
        veh_speed = nusc_can.get_messages(sce, 'vehicle_monitor')
    except:
        print(f'{sce} does not have any CAN bus data!')
        continue
    if len(veh_angle)==0 or len(veh_speed)==0:
        print(f'{sce} have zero CAN bus data!')
        continue

    angles = find_nearest_angle(veh_angle, sorted_samples)
    speeds = find_nearest_speed(veh_speed, sorted_samples)
    # print(f'scene: {sce}, len of angles: {len(angles)}, len of speeds: {len(speeds)}')
    if len(angles)!= len(speeds):
        print(f'scene: {sce}, len of angles: {len(angles)}, len of speeds: {len(speeds)}')
    
    filenames = []
    for sample in sorted_samples:
        token = sample['data']['CAM_FRONT']
        sample_filename = get_sample_filename(token)
        filenames.append(sample_filename)

    meta_data.append(
        {
            'scene': sce,
            'files': filenames,
            'angles': angles,
            'speeds': speeds
        }
    )
len(meta_data)
import json
with open('/mnt/lustrenew/wangxiaodong/data/nuscene/scene_action_file_train.json', 'w') as f:
    json.dump(meta_data, f)

scene: scene-0034, len of angles: 38, len of speeds: 39
scene: scene-0135, len of angles: 38, len of speeds: 39
scene: scene-0138, len of angles: 39, len of speeds: 40
scene: scene-0151, len of angles: 39, len of speeds: 40
scene-0161 does not have any CAN bus data!
scene-0162 does not have any CAN bus data!
scene-0163 does not have any CAN bus data!
scene-0164 does not have any CAN bus data!
scene-0165 does not have any CAN bus data!
scene-0166 does not have any CAN bus data!
scene-0167 does not have any CAN bus data!
scene-0168 does not have any CAN bus data!
scene-0170 does not have any CAN bus data!
scene-0171 does not have any CAN bus data!
scene-0172 does not have any CAN bus data!
scene-0173 does not have any CAN bus data!
scene-0174 does not have any CAN bus data!
scene-0175 does not have any CAN bus data!
scene-0176 does not have any CAN bus data!
scene: scene-0182, len of angles: 39, len of speeds: 40
scene: scene-0194, len of angles: 39, len of speeds: 40
scene: scene-0212, 

/mnt/cachenew2/wangxiaodong/miniconda3/envs/diff/lib/python3.10/site-packages/nuscenes/can_bus/can_bus_api.py:236: UserWarning: Warning: %s does not have any vehicle_monitor messages!
  warnings.warn('Warning: %s does not have any vehicle_monitor messages!')


scene: scene-0430, len of angles: 40, len of speeds: 41
scene: scene-0461, len of angles: 40, len of speeds: 41
scene: scene-0510, len of angles: 40, len of speeds: 41
scene: scene-0512, len of angles: 39, len of speeds: 40
scene: scene-0545, len of angles: 40, len of speeds: 41
scene: scene-0586, len of angles: 40, len of speeds: 41
scene: scene-0687, len of angles: 40, len of speeds: 41
scene: scene-0703, len of angles: 39, len of speeds: 40
scene: scene-0719, len of angles: 39, len of speeds: 40
scene: scene-0809, len of angles: 39, len of speeds: 40
scene: scene-0895, len of angles: 40, len of speeds: 41
scene: scene-0949, len of angles: 39, len of speeds: 40
scene: scene-0953, len of angles: 39, len of speeds: 40
scene: scene-0984, len of angles: 39, len of speeds: 40
scene: scene-0994, len of angles: 39, len of speeds: 40
scene: scene-1005, len of angles: 39, len of speeds: 40


In [13]:
len(meta_data)

684